For a detailed explanation and code interpretation, please visit the [GitHub repository](https://github.com/Mohsto/Loan-requests-prediction).

# Importing libraries

In [41]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Reading the dataset

In [42]:
df = pd.read_csv("loan_data.csv")

In [ ]:
df.head()

In [ ]:
df.info()

Statistical overview

In [ ]:
df.describe()

In [ ]:
df.loc[df["person_age"] > 70]

In [47]:
df = df.loc[~(df["person_age"] > 70)]

Checking for missing data

In [ ]:
df.isna().sum()

# Exploratory data analysis

## Summary statistics

## Univariate analysis

#### Categorical variables

In [ ]:
for category in df.columns:
    if df[category].dtype == 'object':  
        print(f"Value counts for column: {category}")
        print(df[category].value_counts())  
        print("\n")

In [ ]:
df.info()

In [ ]:
names = [
    "Home ownership status", "Purpose of the loan", "Indicator of previous loan defaults",
    "Gender of the person", "Highest education level"
]

categorical_columns = [
    "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file", 
    "person_gender", "person_education"
]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))  
fig.subplots_adjust(hspace = 1.02, wspace = 0.4)  

axes = axes.flatten()

for i, col in enumerate(categorical_columns):
    sns.countplot(x=col, data=df, ax=axes[i])  
    axes[i].set_title(f"Distribution of {names[i]}", fontsize=12)
    axes[i].set_xlabel(names[i])
    axes[i].set_ylabel("Count")
    axes[i].tick_params(axis = 'x', rotation = 90)  

for j in range(len(categorical_columns), len(axes)):
    fig.delaxes(axes[j])

sns.despine()
plt.show()

#### Numerical variables

Distribution and summary

In [ ]:
names = [
    "Person age", "Person income", "Years of employment experience", 
    "Loan amount", "Loan interest rate", "Loan percent income", "Credit score"
]

numeric_columns = [
    "person_age", "person_income", "person_emp_exp", 
    "loan_amnt", "loan_int_rate", "loan_percent_income", "credit_score"
]

fig, axes = plt.subplots(2, 4, figsize=(16, 8)) 
fig.subplots_adjust(hspace = 0.3, wspace = 0.3)  

axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.histplot(df[col], kde = True, bins = 30, ax = axes[i], color = "skyblue")
    axes[i].set_title(f"Distribution of {names[i]}", fontsize = 12)
    axes[i].set_xlabel(names[i])
    axes[i].set_ylabel("Count")

if len(numeric_columns) < len(axes):
    for j in range(len(numeric_columns), len(axes)):
        fig.delaxes(axes[j])

sns.despine()
plt.show()

Boxplots for outliers

In [ ]:
names = [
    "Person age", "Person income", "Years of employment experience", 
    "Loan amount", "Loan interest rate", "Loan percent income", "Credit score"
]

numeric_columns = [
    "person_age", "person_income", "person_emp_exp", 
    "loan_amnt", "loan_int_rate", "loan_percent_income", "credit_score"
]

fig, axes = plt.subplots(2, 4, figsize=(16, 8)) 
fig.subplots_adjust(hspace = 0.3, wspace = 0.3)  

axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.boxplot(df[col], ax = axes[i])
    axes[i].set_title(f"Boxplot of {names[i]}", fontsize = 12)
    axes[i].set_xlabel(names[i])
    axes[i].set_ylabel("")

if len(numeric_columns) < len(axes):
    for j in range(len(numeric_columns), len(axes)):
        fig.delaxes(axes[j])

sns.despine()
plt.show()

## Multivariate analysis

In [ ]:
sns.pairplot(df, hue="loan_status", corner = True, palette="husl")
plt.legend(title = "Loan status", fontsize = 20)

In [ ]:
corr = df.corr(numeric_only = True)
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", vmax = 1, vmin = -1)
plt.title("Correlation Matrix")
plt.show()

## Feature specific analysis

### Persons age

In [56]:
df["person_age_bins"] = pd.cut(
    df["person_age"],
    bins = [19, 25, 30, 40, 71],
    labels = ["20-25", "25-30", "30-40", "40-70"]
)

In [ ]:
print(df["person_age_bins"].value_counts())  
print(df["person_age_bins"].isnull().sum()) 

In [ ]:
grouped_data = df.groupby(["person_age_bins", "loan_status"]).size().reset_index(name = "count")

sns.barplot(
    data = grouped_data,
    x = "person_age_bins",
    y = "count",
    hue = "loan_status", 
    palette = "husl"
)

plt.title("Loan status by age group", fontsize = 16)
plt.xlabel("Age categories")
plt.ylabel("Count")
plt.legend(title = "Loan status")
plt.tight_layout()
sns.despine()
plt.show()

### Loan amount as a percentage of annual income

#### Analyze affordability ratios

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(12, 6))

sns.histplot(
    x='loan_percent_income',
    kde=True, 
    bins=20, 
    palette="muted",
    data=df.query("loan_status == 1"),
    ax=ax[0]
)
ax[0].set_title("Loan status: 1")
ax[0].set_xlabel("Loan percent income")

sns.histplot(
    x='loan_percent_income',
    kde=True, 
    bins=20, 
    palette="muted",
    data=df.query("loan_status == 0"),
    ax=ax[1]
)
ax[1].set_title("Loan status: 0")
ax[1].set_xlabel("Loan percent income")

fig.suptitle("Loan percent of income by status", fontsize=16)

plt.tight_layout()
sns.despine()
plt.show()


### Credit score of the person

In [ ]:
bins = [300, 580, 670, 740, 800, 850]
labels = ["Poor", "Fair", "Good", "Very Good", "Excellent"]

df['credit_score_category'] = pd.cut(df['credit_score'], bins = bins, labels = labels)

sns.countplot(
    x = 'credit_score_category', 
    hue = 'loan_status', 
    data = df
)

plt.title("Credit score category by loan status", fontsize = 16)
plt.legend(title = "Loan status")
sns.despine()
plt.show()

### Loan interest rate

In [ ]:
sns.boxplot(
    x = 'loan_intent', 
    y = 'loan_int_rate', 
    hue = 'loan_status', 
    data = df
)

plt.title("Interest rate by loan intent", fontsize = 16)
plt.xticks(rotation = 45)
plt.xlabel("Loan intent")
plt.ylabel("Loan interest rate")
plt.legend(
    title = "Loan status",
    bbox_to_anchor = (1.2, 1)
)
sns.despine()
plt.show()


### Comparison of loan approval and rejection rates by gender

In [ ]:
gender_loan = df.groupby(["person_gender", "loan_status"]).size().reset_index(name="count")

sns.barplot(
    x="loan_status",
    y="count",
    hue="person_gender",
    data=gender_loan,
    palette="husl"
)

plt.ylabel("Count")
plt.xlabel("Loan status")

handles, labels = plt.gca().get_legend_handles_labels()
labels = [label.capitalize() for label in labels] 
plt.legend(handles, labels, title="Person Gender") 
plt.title("Comparison of loan approval and rejection rates by gender", fontsize = 14, y = 1.05) 

sns.despine()
plt.show()


### Loan status counts by loan intent

In [ ]:
intent_loan = df.groupby(["loan_intent", "loan_status"]).size().reset_index(name = "count")

sns.barplot(
    x = "loan_status",
    y = "count",
    hue = "loan_intent",
    data = intent_loan,
    palette = "husl"
)

plt.ylabel("Count")
plt.xlabel("Loan status")

handles, labels = plt.gca().get_legend_handles_labels()
labels = [label.capitalize() for label in labels] 
plt.legend(handles, labels, title="Loan intent") 
plt.title("Loan status counts by loan intent", fontsize = 16, y = 1.05) 

sns.despine()
plt.show()

### Home ownership vs Loan status

In [ ]:
group_data = df.groupby(['person_home_ownership', 'loan_status']).size().reset_index(name='count')

sns.barplot(
    data = group_data,
    x = 'person_home_ownership', 
    y = 'count', 
    hue = 'loan_status',
    palette = "husl"
)

plt.title("Home ownership vs Loan status", fontsize = 16)
plt.legend(title = "Loan status")
plt.xlabel("")
sns.despine()
plt.show()

### Income by loan status

In [ ]:
sns.boxplot(
    data = df, 
    x = "loan_status", 
    y = "person_income", 
    palette = "Set1"
)

plt.title("Income by loan status", fontsize = 16)
sns.despine()
plt.show()

### Loan amount vs Credit score by loan status

In [ ]:
sns.scatterplot(
    x = 'loan_amnt', 
    y = 'credit_score', 
    hue = 'loan_status', 
    data = df, 
    palette = "husl"
)

plt.title("Loan amount vs Credit score by loan status", fontsize = 16)
sns.despine()
plt.xlabel("Loan amount")
plt.ylabel("Credit score")
plt.legend(
    title = "Loan status",
    bbox_to_anchor = (1.1, 1)
)
plt.show()

# Modeling

In [67]:
df_model = df.copy()

In [ ]:
df_model.head()

## Feature engineering

In [69]:
df_model = pd.get_dummies(df_model, drop_first = True)

bool_columns = df_model.select_dtypes(include='bool').columns

df_model[bool_columns] = df_model[bool_columns].astype(int)

In [ ]:
df_model.head()

## Classification

In [71]:
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [72]:
X = df_model.drop(columns=['loan_status'])
y = df_model['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [73]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

### Logistic regression

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

log_reg = LogisticRegression(random_state = 42)
log_reg_scores = cross_validate(log_reg, X_scaled, y_train, cv = cv, scoring = scoring, return_train_score = False)

print("\nLogistic regression scores:")
for metric in scoring:
    print(f"{metric.capitalize()}: {log_reg_scores['test_' + metric].mean():.4f}")


### Random forest

In [ ]:
random_forest = RandomForestClassifier(random_state = 42)
rf_scores = cross_validate(random_forest, X_scaled, y_train, cv = cv, scoring = scoring, return_train_score = False)

print("\nRandom forest scores:")
for metric in scoring:
    print(f"{metric.capitalize()}: {rf_scores['test_' + metric].mean():.4f}")


In [ ]:
random_forest.fit(X_scaled, y_train)

X_test_scaled = scaler.transform(X_test)

y_pred = random_forest.predict(X_test_scaled)
y_pred_proba = random_forest.predict_proba(X_test_scaled)[:, 1] 

test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\nRandom Forest evaluation on the test data:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print(f"ROC-AUC: {test_roc_auc:.4f}")

### K-nearest neighbors

In [ ]:
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X_train)

knn = KNeighborsClassifier()
knn_scores = cross_validate(knn, X_normalized, y_train, cv = cv, scoring = scoring, return_train_score = False)

print("\nKNN scores:")
for metric in scoring:
    print(f"{metric.capitalize()}: {knn_scores['test_' + metric].mean():.4f}")


### Naive bayes

In [ ]:
naive_bayes = GaussianNB()
nb_scores = cross_validate(naive_bayes, X_scaled, y_train, cv = cv, scoring = scoring, return_train_score = False)

print("\nNaive bayes scores:")
for metric in scoring:
    print(f"{metric.capitalize()}: {nb_scores['test_' + metric].mean():.4f}")